# Data Preparation and Aggregation

This notebook is responsible for cleaning and processing the datasets from the raw data directory. The goal is to prepare the data for visualization in the D3.js web application.

In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

# Define paths - going up from notebooks/ to project root
raw_data_path = '../../data/raw/'
processed_data_path = '../data/processed/'

# Create processed directory if it doesn't exist
os.makedirs(processed_data_path, exist_ok=True)

print("Data preparation starting...")
print(f"Raw data path: {raw_data_path}")
print(f"Processed data path: {processed_data_path}")

Data preparation starting...
Raw data path: ../../data/raw/
Processed data path: ../data/processed/


## 1. Political Advertising Data

We'll process the political ads data to:
- Extract US-focused advertisers only
- Classify advertisers by party (Democratic, Republican, Other)
- Aggregate spending by party over time
- Calculate geographic distribution

In [2]:
# Load political ads weekly spend data
weekly_spend = pd.read_csv(os.path.join(raw_data_path, 'google-political-ads-advertiser-weekly-spend.csv'))
advertiser_stats = pd.read_csv(os.path.join(raw_data_path, 'google-political-ads-advertiser-stats.csv'))

# Preview the data
print(f"Weekly spend records: {len(weekly_spend)}")
print(f"Advertiser stats records: {len(advertiser_stats)}")
print("\nWeekly spend columns:", weekly_spend.columns.tolist())
print("\nAdvertiser stats columns:", advertiser_stats.columns.tolist())
print("\nSample advertiser names:")
print(advertiser_stats['Advertiser_Name'].head(10))

Weekly spend records: 258354
Advertiser stats records: 18657

Weekly spend columns: ['Advertiser_ID', 'Advertiser_Name', 'Election_Cycle', 'Week_Start_Date', 'Spend_USD', 'Spend_EUR', 'Spend_INR', 'Spend_BGN', 'Spend_CZK', 'Spend_DKK', 'Spend_HUF', 'Spend_PLN', 'Spend_RON', 'Spend_SEK', 'Spend_GBP', 'Spend_NZD', 'Spend_ILS', 'Spend_AUD', 'Spend_TWD', 'Spend_BRL', 'Spend_ARS', 'Spend_ZAR', 'Spend_CLP', 'Spend_MXN']

Advertiser stats columns: ['Advertiser_ID', 'Advertiser_Name', 'Public_IDs_List', 'Regions', 'Elections', 'Total_Creatives', 'Spend_USD', 'Spend_EUR', 'Spend_INR', 'Spend_BGN', 'Spend_CZK', 'Spend_DKK', 'Spend_HUF', 'Spend_PLN', 'Spend_RON', 'Spend_SEK', 'Spend_GBP', 'Spend_NZD', 'Spend_ILS', 'Spend_AUD', 'Spend_TWD', 'Spend_BRL', 'Spend_ARS', 'Spend_ZAR', 'Spend_CLP', 'Spend_MXN']

Sample advertiser names:
0                                      SARAH G. NORMAN
1    ELEICAO 2022 RAPHAELA MARIA DE OLIVEIRA MORAES...
2                                    DOUGHTY COMMITTEE
3    

In [3]:
# Function to classify party affiliation based on advertiser name
def classify_party(advertiser_name):
    """
    Classify advertiser by party based on keywords in name
    Returns: 'Democratic', 'Republican', or 'Other'
    """
    if pd.isna(advertiser_name):
        return 'Other'
    
    name_lower = str(advertiser_name).lower()
    
    # Democratic keywords
    dem_keywords = ['democrat', 'biden', 'harris', 'obama', 'clinton', 'pelosi', 
                    'dccc', 'dscc', 'dnc', 'progressive']
    
    # Republican keywords
    rep_keywords = ['republican', 'trump', 'pence', 'mccain', 'romney', 'mcconnell',
                    'nrcc', 'nrsc', 'rnc', 'gop', 'conservative']
    
    # Check for Democratic affiliation
    for keyword in dem_keywords:
        if keyword in name_lower:
            return 'Democratic'
    
    # Check for Republican affiliation
    for keyword in rep_keywords:
        if keyword in name_lower:
            return 'Republican'
    
    return 'Other'

# Apply party classification
advertiser_stats['Party'] = advertiser_stats['Advertiser_Name'].apply(classify_party)

# Check the distribution
print("\nParty distribution:")
print(advertiser_stats['Party'].value_counts())

# Merge party classification with weekly spend data
weekly_spend_with_party = weekly_spend.merge(
    advertiser_stats[['Advertiser_ID', 'Advertiser_Name', 'Party']], 
    on='Advertiser_ID', 
    how='left',
    suffixes=('', '_stats')
)

print(f"\nWeekly spend with party: {len(weekly_spend_with_party)} records")


Party distribution:
Party
Other         17982
Democratic      338
Republican      337
Name: count, dtype: int64

Weekly spend with party: 258354 records


In [4]:
# Filter for US data and convert date
weekly_spend_with_party['Week_Start_Date'] = pd.to_datetime(weekly_spend_with_party['Week_Start_Date'])

# Aggregate by party and week
party_weekly_spend = weekly_spend_with_party.groupby(['Week_Start_Date', 'Party'])['Spend_USD'].sum().reset_index()
party_weekly_spend = party_weekly_spend.sort_values('Week_Start_Date')

# Fill missing party values with 'Other'
party_weekly_spend['Party'] = party_weekly_spend['Party'].fillna('Other')

print("\nAggregated party weekly spend:")
print(party_weekly_spend.head(15))
print(f"\nDate range: {party_weekly_spend['Week_Start_Date'].min()} to {party_weekly_spend['Week_Start_Date'].max()}")

# Save processed political ads data
party_weekly_spend.to_csv(os.path.join(processed_data_path, 'political_ads_party_weekly.csv'), index=False)
print("\n✓ Saved: political_ads_party_weekly.csv")


Aggregated party weekly spend:
   Week_Start_Date       Party  Spend_USD
0       2018-05-27  Democratic       1000
1       2018-05-27       Other       7700
2       2018-05-27  Republican      12900
3       2018-06-03  Democratic       2500
4       2018-06-03       Other      17000
5       2018-06-03  Republican      43000
6       2018-06-10  Democratic       2600
7       2018-06-10       Other      21500
8       2018-06-10  Republican      56800
11      2018-06-17  Republican      54100
9       2018-06-17  Democratic       2800
10      2018-06-17       Other      24400
12      2018-06-24  Democratic       2700
13      2018-06-24       Other      18800
14      2018-06-24  Republican      49600

Date range: 2018-05-27 00:00:00 to 2025-11-16 00:00:00

✓ Saved: political_ads_party_weekly.csv


## 2. User Data Requests

Process government requests for user information from Google.

In [5]:
# Load user data requests
user_data_requests = pd.read_csv(os.path.join(raw_data_path, 'google-global-user-data-requests.csv'))

print(f"User data requests records: {len(user_data_requests)}")
print("\nColumns:", user_data_requests.columns.tolist())
print("\nCountries:", user_data_requests['Country/Region'].unique()[:10])

# Filter for United States only
us_user_requests = user_data_requests[user_data_requests['CLDR Territory Code'] == 'US'].copy()

# Convert period ending to datetime
us_user_requests['Period_Ending'] = pd.to_datetime(us_user_requests['Period Ending'])

# Convert percentage disclosed to numeric (handle string percentages)
us_user_requests['Percentage_Disclosed'] = pd.to_numeric(
    us_user_requests['Percentage disclosed'], errors='coerce'
)

# Select and rename relevant columns
us_user_requests_clean = us_user_requests[[
    'Period_Ending', 'Legal Process', 'User Data Requests', 
    'Percentage_Disclosed', 'Accounts'
]].copy()

# Sort by date
us_user_requests_clean = us_user_requests_clean.sort_values('Period_Ending')

print(f"\nUS user data requests: {len(us_user_requests_clean)} records")
print(f"Date range: {us_user_requests_clean['Period_Ending'].min()} to {us_user_requests_clean['Period_Ending'].max()}")
print("\nLegal processes:", us_user_requests_clean['Legal Process'].unique())

# Aggregate by period (sum across legal process types)
us_user_requests_agg = us_user_requests_clean.groupby('Period_Ending').agg({
    'User Data Requests': 'sum',
    'Accounts': 'sum',
    'Percentage_Disclosed': 'mean'
}).reset_index()

print("\nAggregated US user data requests:")
print(us_user_requests_agg.head(10))

# Save processed user data requests
us_user_requests_agg.to_csv(os.path.join(processed_data_path, 'us_user_data_requests.csv'), index=False)
print("\n✓ Saved: us_user_data_requests.csv")

User data requests records: 4148

Columns: ['Period Ending', 'Country/Region', 'CLDR Territory Code', 'Legal Process', 'User Data Requests', 'Percentage disclosed', 'Accounts']

Countries: ['Argentina' 'Australia' 'Belgium' 'Brazil' 'Canada' 'Chile' 'France'
 'Germany' 'India' 'Israel']

US user data requests: 169 records
Date range: 2009-12-31 00:00:00 to 2024-12-31 00:00:00

Legal processes: ['All' 'Other Legal Requests' 'Search Warrants' 'Subpoenas'
 'Wiretap Orders' 'Pen Register Orders' 'Other Court Orders'
 'Emergency Disclosure Requests' 'Preservation Requests']

Aggregated US user data requests:
  Period_Ending  User Data Requests  Accounts  Percentage_Disclosed
0    2009-12-31              3580.0       0.0              0.000000
1    2010-06-30              4287.0       0.0              0.000000
2    2010-12-31              4601.0       0.0             94.000000
3    2011-06-30              5950.0   11057.0             93.000000
4    2011-12-31              6321.0   12243.0    

## 3. Content Removal Requests

Process government requests to remove content, categorized by reason.

In [6]:
# Load content removal requests (detailed version with reasons)
content_removal = pd.read_csv(os.path.join(raw_data_path, 'google-government-detailed-removal-requests.csv'))

print(f"Content removal requests records: {len(content_removal)}")
print("\nColumns:", content_removal.columns.tolist())

# Filter for United States only
us_content_removal = content_removal[content_removal['CLDR Territory Code'] == 'US'].copy()

# Convert period ending to datetime
us_content_removal['Period_Ending'] = pd.to_datetime(us_content_removal['Period Ending'])

# Select and rename relevant columns
us_content_removal_clean = us_content_removal[[
    'Period_Ending', 'Product', 'Reason', 'Total'
]].copy()

# Sort by date
us_content_removal_clean = us_content_removal_clean.sort_values('Period_Ending')

print(f"\nUS content removal requests: {len(us_content_removal_clean)} records")
print(f"Date range: {us_content_removal_clean['Period_Ending'].min()} to {us_content_removal_clean['Period_Ending'].max()}")
print("\nReasons:", us_content_removal_clean['Reason'].unique())
print("\nProducts:", us_content_removal_clean['Product'].unique())

# Aggregate by period and reason
us_content_removal_by_reason = us_content_removal_clean.groupby(['Period_Ending', 'Reason'])['Total'].sum().reset_index()

print("\nAggregated US content removal by reason:")
print(us_content_removal_by_reason.head(15))

# Save processed content removal data
us_content_removal_by_reason.to_csv(os.path.join(processed_data_path, 'us_content_removal_by_reason.csv'), index=False)
print("\n✓ Saved: us_content_removal_by_reason.csv")

Content removal requests records: 21065

Columns: ['Period Ending', 'Country/Region', 'CLDR Territory Code', 'Product', 'Reason', 'Total']

US content removal requests: 1306 records
Date range: 2011-06-30 00:00:00 to 2025-06-30 00:00:00

Reasons: ['Defamation' 'Violence' 'Privacy and Security' 'Other'
 'National Security' 'Hate Speech' 'Impersonation' 'Copyright'
 'Government Criticism' 'Adult Content' 'Trademark' 'Reason Unspecified'
 'Religious Offense' 'Obscenity/Nudity' 'Bullying/Harassment' 'Drug Abuse'
 'Suicide Promotion' 'Fraud' 'Geographical Dispute' 'Business Complaints'
 'Regulated Goods and Services' 'Electoral Law']

Products: ['Blogger' 'YouTube' 'Web Search' 'Google Images' 'Google Photos'
 'Google Ads' 'Gmail' 'Google Books'
 'Google Earth, Google Maps, and Panoramio' 'Google Groups'
 'Other Google Maps related products' 'Google Sites' 'Google+'
 'Google Apps' 'Panoramio' 'Google Videos' 'AdSense'
 'Google Product Search' 'Google Docs' 'Web Search: Autocomplete'
 'Googl

## 4. Geographic Distribution of Political Ad Spend

Create state-level aggregations for map visualization.

In [7]:
# Load geographic spend data
geo_spend = pd.read_csv(os.path.join(raw_data_path, 'google-political-ads-geo-spend.csv'))

print(f"Geographic spend records: {len(geo_spend)}")
print("\nColumns:", geo_spend.columns.tolist())
print("\nSample regions:")
print(geo_spend['Country_Subdivision_Primary'].unique()[:20])

# Filter for US states (format is like "US-CA", "US-TX", etc.)
us_geo_spend = geo_spend[geo_spend['Country'] == 'US'].copy()

# Extract state code from Country_Subdivision_Primary
us_geo_spend['State'] = us_geo_spend['Country_Subdivision_Primary'].str.replace('US-', '')

# Aggregate spend by state
state_spend = us_geo_spend.groupby('State')['Spend_USD'].sum().reset_index()
state_spend = state_spend.sort_values('Spend_USD', ascending=False)

print("\nTop 10 states by political ad spend:")
print(state_spend.head(10))

# Save processed geographic data
state_spend.to_csv(os.path.join(processed_data_path, 'political_ads_state_spend.csv'), index=False)
print("\n✓ Saved: political_ads_state_spend.csv")

Geographic spend records: 1108

Columns: ['Country', 'Country_Subdivision_Primary', 'Country_Subdivision_Secondary', 'Spend_USD', 'Spend_EUR', 'Spend_INR', 'Spend_BGN', 'Spend_CZK', 'Spend_DKK', 'Spend_HUF', 'Spend_PLN', 'Spend_RON', 'Spend_SEK', 'Spend_GBP', 'Spend_NZD', 'Spend_ILS', 'Spend_AUD', 'Spend_TWD', 'Spend_BRL', 'Spend_ARS', 'Spend_ZAR', 'Spend_CLP', 'Spend_MXN']

Sample regions:
['US-MD' 'US-TX' 'US-CT' 'MX-GRO' 'ZA-NL' 'US-PA' 'US-MA' 'US-CA' 'US-NY'
 'IN-KL' 'US-FL' 'US-LA' 'US-KY' 'US-AL' 'US-IL' nan 'US-WI' 'MX-COL'
 'CL-BI' 'US-GA']

Top 10 states by political ad spend:
   State  Spend_USD
5     CA  244804100
41    PA  189263600
24    MI  140625500
11    GA  137286900
10    FL  113190700
53    WI  112818600
4     AZ  104024300
30    NC  100889400
47    TX   96350600
38    OH   83026100

✓ Saved: political_ads_state_spend.csv


## 5. Summary Statistics

Display key insights from the processed datasets.

In [8]:
print("=" * 80)
print("DATA PROCESSING COMPLETE")
print("=" * 80)

print("\n📊 POLITICAL ADVERTISING")
print(f"   Total spend by party:")
party_totals = party_weekly_spend.groupby('Party')['Spend_USD'].sum().sort_values(ascending=False)
for party, spend in party_totals.items():
    print(f"   {party}: ${spend:,.0f}")

print(f"\n   Time range: {party_weekly_spend['Week_Start_Date'].min().date()} to {party_weekly_spend['Week_Start_Date'].max().date()}")
print(f"   Total weeks: {len(party_weekly_spend['Week_Start_Date'].unique())}")

print("\n🔍 USER DATA REQUESTS")
print(f"   Total requests: {us_user_requests_agg['User Data Requests'].sum():,.0f}")
print(f"   Total accounts affected: {us_user_requests_agg['Accounts'].sum():,.0f}")
print(f"   Average disclosure rate: {us_user_requests_agg['Percentage_Disclosed'].mean():.1f}%")
print(f"   Time range: {us_user_requests_agg['Period_Ending'].min().date()} to {us_user_requests_agg['Period_Ending'].max().date()}")

print("\n🚫 CONTENT REMOVAL REQUESTS")
print(f"   Total removal requests: {us_content_removal_by_reason['Total'].sum():,.0f}")
print(f"   Top reasons:")
reason_totals = us_content_removal_by_reason.groupby('Reason')['Total'].sum().sort_values(ascending=False).head(5)
for reason, count in reason_totals.items():
    print(f"   {reason}: {count:,.0f}")
print(f"   Time range: {us_content_removal_by_reason['Period_Ending'].min().date()} to {us_content_removal_by_reason['Period_Ending'].max().date()}")

print("\n📁 PROCESSED FILES CREATED:")
print("   ✓ political_ads_party_weekly.csv")
print("   ✓ us_user_data_requests.csv")
print("   ✓ us_content_removal_by_reason.csv")
print("   ✓ political_ads_state_spend.csv")

print("\n" + "=" * 80)

DATA PROCESSING COMPLETE

📊 POLITICAL ADVERTISING
   Total spend by party:
   Other: $1,558,748,800
   Democratic: $358,666,900
   Republican: $285,281,200

   Time range: 2018-05-27 to 2025-11-16
   Total weeks: 391

🔍 USER DATA REQUESTS
   Total requests: 1,063,243
   Total accounts affected: 2,279,198
   Average disclosure rate: 77.9%
   Time range: 2009-12-31 to 2024-12-31

🚫 CONTENT REMOVAL REQUESTS
   Total removal requests: 12,564
   Top reasons:
   Defamation: 6,731
   Privacy and Security: 1,346
   Fraud: 1,005
   Bullying/Harassment: 959
   Trademark: 943
   Time range: 2011-06-30 to 2025-06-30

📁 PROCESSED FILES CREATED:
   ✓ political_ads_party_weekly.csv
   ✓ us_user_data_requests.csv
   ✓ us_content_removal_by_reason.csv
   ✓ political_ads_state_spend.csv

